# monitors

> Client-side SSE connection monitoring and management utilities

In [ ]:
#| default_exp components.monitors

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Dict, Any, List, Callable
from dataclasses import dataclass, field
from enum import Enum
import json

## Connection States

In [ ]:
#| export
class ConnectionState(Enum):
    """SSE connection states"""
    CONNECTING = "connecting"
    CONNECTED = "connected"
    DISCONNECTED = "disconnected"
    ERROR = "error"
    RECONNECTING = "reconnecting"
    CLOSED = "closed"

## Monitor Configuration

In [ ]:
#| export
@dataclass
class MonitorConfig:
    """
    Configuration for SSE connection monitoring.
    
    Attributes:
        sse_element_id: ID of the SSE connection element to monitor
        status_element_id: ID of element to update with connection status
        auto_reconnect: Whether to automatically reconnect on failures
        reconnect_delay: Delay in ms before reconnection attempts
        max_reconnect_attempts: Maximum number of reconnection attempts (None = infinite)
        visibility_reconnect: Reconnect when tab becomes visible
        heartbeat_timeout: Timeout in ms to consider connection dead (None = no timeout)
        debug: Enable console logging
        callbacks: JavaScript callbacks for state changes
    """
    sse_element_id: str
    status_element_id: Optional[str] = None
    auto_reconnect: bool = True
    reconnect_delay: int = 3000
    max_reconnect_attempts: Optional[int] = None
    visibility_reconnect: bool = True
    heartbeat_timeout: Optional[int] = None
    debug: bool = False
    callbacks: Dict[str, str] = field(default_factory=dict)
    
    def to_js_config(
        self
    ) -> str:  # JSON string of the JavaScript configuration object
        """Convert to JavaScript configuration object"""
        config = {
            "sseElementId": self.sse_element_id,
            "statusElementId": self.status_element_id,
            "autoReconnect": self.auto_reconnect,
            "reconnectDelay": self.reconnect_delay,
            "maxReconnectAttempts": self.max_reconnect_attempts,
            "visibilityReconnect": self.visibility_reconnect,
            "heartbeatTimeout": self.heartbeat_timeout,
            "debug": self.debug
        }
        # Remove None values
        config = {k: v for k, v in config.items() if v is not None}
        return json.dumps(config)

## JavaScript Monitor Generator

In [ ]:
#| export
def generate_monitor_script(
    config: MonitorConfig,  # Monitor configuration
    status_indicators: Optional[Dict[ConnectionState, str]] = None
) -> str:  # JavaScript code as a string
    "Generate JavaScript for monitoring SSE connections."
    # Default status indicators if not provided
    if status_indicators is None:
        status_indicators = {
            ConnectionState.CONNECTING: "Connecting...",
            ConnectionState.CONNECTED: "Connected",
            ConnectionState.DISCONNECTED: "Disconnected",
            ConnectionState.ERROR: "Connection Error",
            ConnectionState.RECONNECTING: "Reconnecting...",
            ConnectionState.CLOSED: "Connection Closed"
        }
    
    # Convert indicators to JavaScript object
    indicators_js = json.dumps({
        state.value: html for state, html in status_indicators.items()
    })
    
    # Generate JavaScript
    return f"""
(function() {{
    const config = {config.to_js_config()};
    const statusIndicators = {indicators_js};
    
    class SSEMonitor {{
        constructor(cfg) {{
            this.config = cfg;
            this.element = null;
            this.statusElement = null;
            this.reconnectAttempts = 0;
            this.reconnectTimer = null;
            this.heartbeatTimer = null;
            this.lastHeartbeat = Date.now();
            this.state = '{ConnectionState.DISCONNECTED.value}';
            
            this.init();
        }}
        
        log(message) {{
            if (this.config.debug) {{
                console.log('[SSE Monitor] ' + message);
            }}
        }}
        
        updateStatus(state) {{
            this.state = state;
            this.log('State: ' + state);
            
            if (this.statusElement && statusIndicators[state]) {{
                this.statusElement.innerHTML = statusIndicators[state];
            }}
            
            // Trigger custom callback if defined
            const callback = this.config.callbacks && this.config.callbacks[state];
            if (callback && typeof window[callback] === 'function') {{
                window[callback](state);
            }}
        }}
        
        init() {{
            this.element = document.getElementById(this.config.sseElementId);
            if (!this.element) {{
                this.log('SSE element not found: ' + this.config.sseElementId);
                return;
            }}
            
            if (this.config.statusElementId) {{
                this.statusElement = document.getElementById(this.config.statusElementId);
            }}
            
            this.attachListeners();
            this.setupVisibilityHandling();
            this.setupHeartbeatMonitor();
        }}
        
        attachListeners() {{
            // HTMX SSE events
            this.element.addEventListener('htmx:sseOpen', (evt) => {{
                this.log('SSE connection opened');
                this.updateStatus('{ConnectionState.CONNECTED.value}');
                this.reconnectAttempts = 0;
                this.resetHeartbeat();
            }});
            
            this.element.addEventListener('htmx:sseClose', (evt) => {{
                this.log('SSE connection closed');
                this.updateStatus('{ConnectionState.CLOSED.value}');
                this.handleDisconnection();
            }});
            
            this.element.addEventListener('htmx:sseError', (evt) => {{
                this.log('SSE connection error');
                this.updateStatus('{ConnectionState.ERROR.value}');
                this.handleDisconnection();
            }});
            
            // Track SSE messages for heartbeat
            this.element.addEventListener('sse:message', (evt) => {{
                this.resetHeartbeat();
            }});
        }}
        
        setupVisibilityHandling() {{
            if (!this.config.visibilityReconnect) return;
            
            document.addEventListener('visibilitychange', () => {{
                if (!document.hidden) {{
                    this.log('Tab became visible, checking connection');
                    this.checkConnection();
                }}
            }});
        }}
        
        setupHeartbeatMonitor() {{
            if (!this.config.heartbeatTimeout) return;
            
            setInterval(() => {{
                const timeSinceLastHeartbeat = Date.now() - this.lastHeartbeat;
                if (timeSinceLastHeartbeat > this.config.heartbeatTimeout) {{
                    this.log('Heartbeat timeout, connection may be dead');
                    this.handleDisconnection();
                }}
            }}, this.config.heartbeatTimeout / 2);
        }}
        
        resetHeartbeat() {{
            this.lastHeartbeat = Date.now();
        }}
        
        checkConnection() {{
            const sseData = this.element['htmx-internal-data'];
            if (!sseData || !sseData.sseEventSource) {{
                this.log('No SSE connection found');
                this.reconnect();
                return;
            }}
            
            const state = sseData.sseEventSource.readyState;
            switch(state) {{
                case EventSource.CONNECTING:
                    this.updateStatus('{ConnectionState.CONNECTING.value}');
                    break;
                case EventSource.OPEN:
                    this.updateStatus('{ConnectionState.CONNECTED.value}');
                    break;
                case EventSource.CLOSED:
                    this.handleDisconnection();
                    break;
            }}
        }}
        
        handleDisconnection() {{
            if (this.config.autoReconnect && this.canReconnect()) {{
                this.scheduleReconnect();
            }} else {{
                this.updateStatus('{ConnectionState.DISCONNECTED.value}');
            }}
        }}
        
        canReconnect() {{
            if (!this.config.maxReconnectAttempts) return true;
            return this.reconnectAttempts < this.config.maxReconnectAttempts;
        }}
        
        scheduleReconnect() {{
            if (this.reconnectTimer) return;
            
            this.updateStatus('{ConnectionState.RECONNECTING.value}');
            this.reconnectTimer = setTimeout(() => {{
                this.reconnectTimer = null;
                this.reconnect();
            }}, this.config.reconnectDelay);
        }}
        
        reconnect() {{
            this.reconnectAttempts++;
            this.log('Reconnection attempt ' + this.reconnectAttempts);
            
            // Trigger HTMX to reconnect
            if (typeof htmx !== 'undefined') {{
                htmx.trigger(this.element, 'htmx:sseReconnect');
            }}
        }}
        
        destroy() {{
            if (this.reconnectTimer) {{
                clearTimeout(this.reconnectTimer);
            }}
            if (this.heartbeatTimer) {{
                clearInterval(this.heartbeatTimer);
            }}
        }}
    }}
    
    // Initialize monitor when DOM is ready
    if (document.readyState === 'loading') {{
        document.addEventListener('DOMContentLoaded', () => {{
            window.sseMonitor = new SSEMonitor(config);
        }});
    }} else {{
        window.sseMonitor = new SSEMonitor(config);
    }}
}})();
"""

## Simple Monitor Functions

In [ ]:
#| export
def generate_simple_monitor(
    sse_element_id: str,  # ID of SSE element to monitor
    status_element_id: Optional[str] = None,  # Optional status display element
    debug: bool = False  # Enable console logging
) -> str:  # JavaScript code for basic monitoring
    "Generate a simple SSE connection monitor."
    config = MonitorConfig(
        sse_element_id=sse_element_id,
        status_element_id=status_element_id,
        debug=debug,
        auto_reconnect=True,
        visibility_reconnect=True
    )
    return generate_monitor_script(config)

In [ ]:
#| export
def generate_connection_counter(
) -> str:  # JavaScript code that tracks connection count
    "Generate JavaScript for counting active SSE connections."
    return """
(function() {
    window.SSEConnectionCounter = {
        connections: new Map(),
        
        add(elementId) {
            this.connections.set(elementId, 'active');
            this.updateCount();
        },
        
        remove(elementId) {
            this.connections.delete(elementId);
            this.updateCount();
        },
        
        updateCount() {
            const count = this.connections.size;
            document.dispatchEvent(new CustomEvent('sse:connectionCountChanged', {
                detail: { count: count }
            }));
        },
        
        getCount() {
            return this.connections.size;
        },
        
        init() {
            document.querySelectorAll('[sse-connect]').forEach(element => {
                const id = element.id || element.getAttribute('sse-connect');
                
                element.addEventListener('htmx:sseOpen', () => {
                    this.add(id);
                });
                
                element.addEventListener('htmx:sseClose', () => {
                    this.remove(id);
                });
                
                element.addEventListener('htmx:sseError', () => {
                    this.remove(id);
                });
            });
        }
    };
    
    // Initialize when DOM is ready
    if (document.readyState === 'loading') {
        document.addEventListener('DOMContentLoaded', () => {
            SSEConnectionCounter.init();
        });
    } else {
        SSEConnectionCounter.init();
    }
})();
""".strip()

## Example Usage

In [ ]:
# Example: Creating a monitor configuration
def example_monitor_config():
    config = MonitorConfig(
        sse_element_id="global-sse",
        status_element_id="connection-status",
        auto_reconnect=True,
        reconnect_delay=5000,
        max_reconnect_attempts=10,
        visibility_reconnect=True,
        debug=True
    )
    
    print("Monitor Configuration:")
    print(f"  SSE Element: {config.sse_element_id}")
    print(f"  Status Element: {config.status_element_id}")
    print(f"  Auto Reconnect: {config.auto_reconnect}")
    print(f"  Max Attempts: {config.max_reconnect_attempts}")
    print()
    
    print("JavaScript config:")
    js_config = config.to_js_config()
    print(js_config)
    print()
    
    # Generate script (show first 500 chars)
    script = generate_monitor_script(config)
    print(f"Generated script (first 500 chars):")
    print(script[:500] + "...")

example_monitor_config()

Monitor Configuration:
  SSE Element: global-sse
  Status Element: connection-status
  Auto Reconnect: True
  Max Attempts: 10

JavaScript config:
{"sseElementId": "global-sse", "statusElementId": "connection-status", "autoReconnect": true, "reconnectDelay": 5000, "maxReconnectAttempts": 10, "visibilityReconnect": true, "debug": true}

Generated script (first 500 chars):

(function() {
    const config = {"sseElementId": "global-sse", "statusElementId": "connection-status", "autoReconnect": true, "reconnectDelay": 5000, "maxReconnectAttempts": 10, "visibilityReconnect": true, "debug": true};
    const statusIndicators = {"connecting": "Connecting...", "connected": "Connected", "disconnected": "Disconnected", "error": "Connection Error", "reconnecting": "Reconnecting...", "closed": "Connection Closed"};

    class SSEMonitor {
        constructor(cfg) {
         ...


In [ ]:
# Example: Simple monitor generation
def example_simple_monitor():
    script = generate_simple_monitor(
        sse_element_id="my-sse-connection",
        status_element_id="status-display",
        debug=False
    )
    
    print("Simple monitor script generated")
    print(f"Script length: {len(script)} characters")
    print()
    
    # Show configuration section
    config_start = script.find("const config = ")
    config_end = script.find(";", config_start) + 1
    print("Configuration section:")
    print(script[config_start:config_end])

example_simple_monitor()

Simple monitor script generated
Script length: 6228 characters

Configuration section:
const config = {"sseElementId": "my-sse-connection", "statusElementId": "status-display", "autoReconnect": true, "reconnectDelay": 3000, "visibilityReconnect": true, "debug": false};


In [ ]:
# Example: Connection counter
def example_connection_counter():
    counter_script = generate_connection_counter()
    
    print("Connection Counter Script:")
    print(f"Script length: {len(counter_script)} characters")
    print()
    
    # Show the main object definition
    obj_start = counter_script.find("window.SSEConnectionCounter = {")
    obj_end = counter_script.find("add(elementId) {", obj_start) + 20
    print("Object definition (partial):")
    print(counter_script[obj_start:obj_end] + "...")
    print()
    
    print("Features:")
    print("  - Tracks active connections")
    print("  - Dispatches custom events on count changes")
    print("  - Auto-initializes on DOM ready")

example_connection_counter()

Connection Counter Script:
Script length: 1458 characters

Object definition (partial):
window.SSEConnectionCounter = {
        connections: new Map(),

        add(elementId) {
   ...

Features:
  - Tracks active connections
  - Dispatches custom events on count changes
  - Auto-initializes on DOM ready


## Testing

In [ ]:
# Test: MonitorConfig
def test_monitor_config():
    config = MonitorConfig(
        sse_element_id="test-sse",
        status_element_id="test-status",
        reconnect_delay=5000,
        max_reconnect_attempts=5
    )
    
    js = config.to_js_config()
    js_obj = json.loads(js)
    
    assert js_obj["sseElementId"] == "test-sse"
    assert js_obj["statusElementId"] == "test-status"
    assert js_obj["reconnectDelay"] == 5000
    assert js_obj["maxReconnectAttempts"] == 5
    assert js_obj["autoReconnect"] == True
    
    print("✓ MonitorConfig tests passed")

test_monitor_config()

✓ MonitorConfig tests passed


In [ ]:
# Test: Script generation
def test_script_generation():
    config = MonitorConfig(
        sse_element_id="test",
        debug=True
    )
    
    script = generate_monitor_script(config)
    
    # Check that script contains key components
    assert "SSEMonitor" in script
    assert "htmx:sseOpen" in script
    assert "htmx:sseClose" in script
    assert "htmx:sseError" in script
    assert "visibilitychange" in script
    assert "reconnect" in script
    
    # Check simple monitor
    simple = generate_simple_monitor("test-id")
    assert "test-id" in simple
    assert "SSEMonitor" in simple
    
    print("✓ Script generation tests passed")

test_script_generation()

✓ Script generation tests passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()